In [ ]:
# from bluesky_queueserver_api.zmq import REManagerAPI

RM = REManagerAPI(zmq_control_addr="tcp://isnpc01.xray.aps.anl.gov:60615", zmq_info_addr="tcp://isnpc01.xray.aps.anl.gov:60625")

In [ ]:


import sys
import time
from pvaccess import Channel
from collections import OrderedDict

class ClientMonitor:

    def __init__(self, name):
        self.name = name
        self.value = 0
        self.nReceived = 0
        self.nMissed = 0
        self.percentageMissed = 0
        self.startTime = 0
        self.receiveRate = 0

    def toString(self):
        return '%6s: Received: %7d (%6.2f [kHz]); Missed: %7d (%6.2f%%)' % (self.name, self.nReceived, self.receiveRateKHz, self.nMissed, self.percentageMissed)
       
    def monitor(self, pv):
        oldValue = self.value
        self.value = pv['c']
        self.nReceived += 1
        diff = self.value - oldValue
        if oldValue > 0:
            self.nMissed += diff-1
        else:
            self.startTime = time.time()
 
        if self.nReceived % 10000 == 0:
            currentTime = time.time()
            deltaT = currentTime - self.startTime
            self.receiveRateKHz = self.nReceived/deltaT/1000.0
            self.percentageMissed = (self.nMissed*100.0)/(self.nReceived+self.nMissed)
            
        if self.nReceived % 100000 == 0:
            print(self.toString())

if __name__ == '__main__':
    runtime = 60
    if len(sys.argv) > 1:
        runtime = float(sys.argv[1])

    channelName = 'XSP3_1Chan:Pva1:Image'
    c = Channel(channelName)
    #print('CONNECT TO %s:\n%s\n' % (channelName, c.get()))
    m = ClientMonitor(channelName)

    t0 = time.time()
    print('STARTING MONITOR for %s at %s\n' % (channelName, t0))
    #c.monitor(m.monitor)
    c.monitor(m.monitor, 'field(c)')
    time.sleep(runtime)
    c.stopMonitor()
    t1 = time.time()
    deltaT = t1-t0
    print('STOP MONITOR at %s\n' % t1)

    print('FINAL STATS:') 
    print(m.toString())
    print('')
    print('RUNTIME: %.2f [s]' % (deltaT))
    print('\nDONE')


: 

In [6]:
status = RM.send_request(method="status")
status

{'msg': 'RE Manager v0.0.19',
 'items_in_queue': 1,
 'items_in_history': 2,
 'running_item_uid': None,
 'manager_state': 'idle',
 'queue_stop_pending': False,
 'queue_autostart_enabled': False,
 'worker_environment_exists': True,
 'worker_environment_state': 'idle',
 'worker_background_tasks': 0,
 're_state': 'idle',
 'ip_kernel_state': 'disabled',
 'ip_kernel_captured': True,
 'pause_pending': False,
 'run_list_uid': '77cdd75d-f56c-409c-a866-a5c6b2287c72',
 'plan_queue_uid': '943fe7fd-3642-47ab-a773-f5f7e81ec101',
 'plan_history_uid': '0abddd52-43e2-42a3-8fba-9decdac65d15',
 'devices_existing_uid': '499dd237-09da-4d6c-b1ac-e0fd891b67f5',
 'plans_existing_uid': 'dc0fe0ec-7fd7-4276-95e4-973ba4ae5594',
 'devices_allowed_uid': '4e4013b1-a1c6-4250-acb6-67a9e2f43406',
 'plans_allowed_uid': '72226278-9784-4652-9c1c-365a2a4ea3e1',
 'plan_queue_mode': {'loop': False, 'ignore_failures': False},
 'task_results_uid': '7a1cad46-7080-4822-966c-1928820d743b',
 'lock_info_uid': 'bc1ba136-5bfb-4b01-87